In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sqlalchemy import create_engine, inspect
from utils.database_utils import *
from utils.config_utils import *
from constants import *

db_config = get_db_config(read_config(CONFIG_FILE_PATH))
# Create SQLAlchemy engine
engine = connect(db_config)

# Connect and Inspect
inspector = inspect(engine)

# Get table names
table_names = inspector.get_table_names()
table_names

Connection to PostgreSQL successful


['truck_schedule_table',
 'routes_table',
 'routes_weather',
 'trucks_table',
 'traffic_table',
 'drivers_table',
 'city_weather']

In [3]:
import hopsworks
import pandas as pd

project = hopsworks.login(api_key_value = 'HIW4D6r4iHGmgSqT.W5LdkqjulcZ85kFXhePJdfJb2sxxZigzNOtweKlAXTLiou7QVkWPBzc9gfRXMjYC')
fs = project.get_feature_store()

feature_groups = ['final_merge_fg']

feature_dataframes = {}

# Loop through each feature group and fetch the data
for fg_name in feature_groups:
    # Get the feature group object
    fg = fs.get_feature_group(fg_name, version=1)  # Ensure you use the correct version
    
    # Fetch the data as a Pandas DataFrame
    query = fg.select_all()
    df=query.read(read_options={"use_hive": True})
    # df = fg.read(read_options={"use_hive": True})
    
    # Store in dictionary
    feature_dataframes[fg_name] = df

# Now you have a dictionary of DataFrames, where the key is the feature group name
print(feature_dataframes)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1022104
Connected. Call `.close()` to terminate connection gracefully.
2024-10-22 23:14:23,959 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (218.27s) 
{'final_merge_fg':        unique_id  truck_id    route_id      departure_date  \
0           9778  22007337  R-b69ad412 2019-01-16 07:00:00   
1           1566  12052665  R-9e7d29c6 2019-01-07 07:00:00   
2           7277  57473533  R-ef8dd68d 2019-02-03 07:00:00   
3           9026  99047862  R-58474583 2019-02-12 07:00:00   
4           2016  19445339  R-d52fc715 2019-01-10 07:00:00   
...          ...       ...         ...                 ...   
10516       4237  43346401  R-089c9405 2019-01-19 07:00:00   
10517       3517  22748353  R-8656be91 2019-01-16 07:00:00   
10518      10526  29066789  R-0810e90f 2019-02-10 07:00:00   
10519    

In [4]:
for key, df in feature_dataframes.items():
    if 'event_time' in df.columns:
        df.drop(columns=['event_time'], inplace=True)

In [5]:
final_merge = feature_dataframes['final_merge_fg']

In [6]:
final_merge.head()

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,9778,22007337,R-b69ad412,2019-01-16 07:00:00,2019-01-17 11:14:24,1,45.333333,14.0,0.0,92.000000,...,1aabc91a-c,Ricardo Pena,male,50.0,9.0,conservative,7.0,22007337.0,44.91,1
1,1566,12052665,R-9e7d29c6,2019-01-07 07:00:00,2019-01-07 14:26:24,0,43.333333,10.0,0.0,65.666667,...,45b11b01-6,Greg Gallagher,male,40.0,2.0,conservative,2.0,12052665.0,43.79,0
2,7277,57473533,R-ef8dd68d,2019-02-03 07:00:00,2019-02-04 05:19:12,1,46.200000,4.6,0.0,57.600000,...,fde76a0f-7,Dr. Chad Garza,male,40.0,5.0,proactive,3.0,57473533.0,64.71,1
3,9026,99047862,R-58474583,2019-02-12 07:00:00,2019-02-12 14:32:24,0,62.333333,7.0,0.0,68.000000,...,b722825b-3,Matthew Allison,male,59.0,18.0,conservative,6.0,99047862.0,44.91,0
4,2016,19445339,R-d52fc715,2019-01-10 07:00:00,2019-01-10 08:22:48,0,73.000000,4.5,0.0,93.500000,...,95f5db58-1,Dr. Gregory Lutz,male,45.0,1.0,conservative,8.0,19445339.0,48.44,0


In [7]:
cts_cols=['route_avg_temp', 'route_avg_wind_speed',
              'route_avg_precip', 'route_avg_humidity', 'route_avg_visibility',
              'route_avg_pressure', 'distance', 'average_hours',
              'origin_temp', 'origin_wind_speed', 'origin_precip', 'origin_humidity',
              'origin_visibility', 'origin_pressure',
              'destination_temp','destination_wind_speed','destination_precip',
              'destination_humidity', 'destination_visibility','destination_pressure',
               'avg_no_of_vehicles', 'truck_age','load_capacity_pounds', 'mileage_mpg',
               'age', 'experience','average_speed_mph']
       
       
cat_cols=['route_description',
        'origin_description', 'destination_description',
        'accident', 'fuel_type',
        'gender', 'driving_style', 'ratings','is_midnight']


target=['delay']

In [8]:
final_merge['estimated_arrival'].min(), final_merge['estimated_arrival'].max()

(Timestamp('2019-01-01 07:04:48'), Timestamp('2019-02-14 10:01:48'))

In [11]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(final_merge, test_size=0.4, random_state=42, shuffle=True)

# Step 2: Split the remaining 40% into 20% validation and 20% test
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

In [21]:
train_df = final_merge[final_merge['estimated_arrival'] <= pd.to_datetime('2019-01-30')]

validation_df = final_merge[(final_merge['estimated_arrival'] > pd.to_datetime('2019-01-30')) &

                            (final_merge['estimated_arrival'] <= pd.to_datetime('2019-02-07'))]

test_df = final_merge[final_merge['estimated_arrival'] > pd.to_datetime('2019-02-07')]

In [12]:
X_train=train_df[cts_cols+cat_cols]
y_train=train_df['delay']

X_valid = validation_df[cts_cols + cat_cols]
y_valid = validation_df['delay']

X_test=test_df[cts_cols+cat_cols]
y_test=test_df['delay']

In [13]:
X_train

,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,distance,average_hours,origin_temp,origin_wind_speed,...,average_speed_mph,route_description,origin_description,destination_description,accident,fuel_type,gender,driving_style,ratings,is_midnight
1867,63.666667,10.000000,0.0,72.333333,6.000000,1007.333333,297.54,5.95,39.0,11.0,...,35.32,Light rain shower,Sunny,Sunny,0,gas,male,conservative,4.0,0
10428,74.000000,10.000000,0.0,49.333333,6.000000,1013.000000,314.05,6.28,30.0,9.0,...,54.29,Partly cloudy,Patchy moderate snow,Sunny,0,diesel,male,conservative,6.0,0
4597,70.000000,10.333333,0.0,55.666667,6.000000,1007.666667,274.13,5.48,25.0,12.0,...,60.31,Sunny,Overcast,Heavy snow,0,diesel,male,proactive,7.0,0
2521,53.500000,6.500000,0.0,48.750000,6.000000,1019.000000,833.74,16.67,46.0,2.0,...,49.74,Clear,Fog,Overcast,1,diesel,male,conservative,8.0,0
4952,52.000000,5.000000,0.0,70.500000,6.000000,1019.000000,126.12,2.52,57.0,11.0,...,64.11,Sunny,Patchy light drizzle,Sunny,0,diesel,male,proactive,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,81.500000,8.250000,0.0,77.250000,5.750000,1012.000000,793.61,15.87,25.0,14.0,...,47.94,Patchy rain possible,Overcast,Moderate or heavy rain shower,1,diesel,male,conservative,3.0,0
5191,63.750000,1.250000,0.0,91.250000,6.000000,1012.750000,1943.16,38.86,27.0,3.0,...,40.98,Patchy rain possible,Partly cloudy,Clear,1,gas,female,conservative,4.0,1
5390,63.000000,9.600000,0.0,78.800000,6.000000,1015.400000,1103.33,22.07,32.0,6.0,...,63.95,Sunny,Overcast,Light drizzle,1,gas,male,proactive,3.0,1
860,48.000000,7.000000,0.0,80.333333,5.166667,1020.666667,1432.97,28.66,66.0,6.0,...,61.30,Partly cloudy,Fog,Sunny,1,gas,male,proactive,6.0,1


In [14]:
from sklearn.preprocessing import OneHotEncoder
encoder_columns = ['route_description', 'origin_description', 'destination_description', 'fuel_type', 'gender', 'driving_style']

encoder = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
encoder.fit(X_train[encoder_columns])
encoded_features = list(encoder.get_feature_names_out(encoder_columns))

In [15]:
X_train[encoded_features] = encoder.transform(X_train[encoder_columns])

X_valid[encoded_features] = encoder.transform(X_valid[encoder_columns])

X_test[encoded_features] = encoder.transform(X_test[encoder_columns])

2024-10-22 23:20:33,925 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-10-22 23:20:33,933 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-10-22 23:20:33,936 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-10-22 23:20:33,936 WARNING: S

In [16]:
X_train

,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,distance,average_hours,origin_temp,origin_wind_speed,...,destination_description_Patchy snow possible,destination_description_Sunny,destination_description_Thundery outbreaks possible,destination_description_Torrential rain shower,fuel_type_diesel,fuel_type_gas,gender_female,gender_male,driving_style_conservative,driving_style_proactive
1867,63.666667,10.000000,0.0,72.333333,6.000000,1007.333333,297.54,5.95,39.0,11.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
10428,74.000000,10.000000,0.0,49.333333,6.000000,1013.000000,314.05,6.28,30.0,9.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4597,70.000000,10.333333,0.0,55.666667,6.000000,1007.666667,274.13,5.48,25.0,12.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2521,53.500000,6.500000,0.0,48.750000,6.000000,1019.000000,833.74,16.67,46.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4952,52.000000,5.000000,0.0,70.500000,6.000000,1019.000000,126.12,2.52,57.0,11.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,81.500000,8.250000,0.0,77.250000,5.750000,1012.000000,793.61,15.87,25.0,14.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
5191,63.750000,1.250000,0.0,91.250000,6.000000,1012.750000,1943.16,38.86,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
5390,63.000000,9.600000,0.0,78.800000,6.000000,1015.400000,1103.33,22.07,32.0,6.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
860,48.000000,7.000000,0.0,80.333333,5.166667,1020.666667,1432.97,28.66,66.0,6.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [17]:
#LAbel encoding
from sklearn.preprocessing import LabelEncoder
laben = LabelEncoder()
label_encoder_columns = ['accident', 'ratings','is_midnight']
for col in label_encoder_columns:
    X_train[col] = laben.fit_transform(X_train[col])


In [18]:
X_train = X_train.drop(encoder_columns, axis=1)

X_valid = X_valid.drop(encoder_columns, axis=1)

X_test = X_test.drop(encoder_columns, axis=1)

In [19]:
X_train

,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,distance,average_hours,origin_temp,origin_wind_speed,...,destination_description_Patchy snow possible,destination_description_Sunny,destination_description_Thundery outbreaks possible,destination_description_Torrential rain shower,fuel_type_diesel,fuel_type_gas,gender_female,gender_male,driving_style_conservative,driving_style_proactive
1867,63.666667,10.000000,0.0,72.333333,6.000000,1007.333333,297.54,5.95,39.0,11.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
10428,74.000000,10.000000,0.0,49.333333,6.000000,1013.000000,314.05,6.28,30.0,9.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4597,70.000000,10.333333,0.0,55.666667,6.000000,1007.666667,274.13,5.48,25.0,12.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2521,53.500000,6.500000,0.0,48.750000,6.000000,1019.000000,833.74,16.67,46.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4952,52.000000,5.000000,0.0,70.500000,6.000000,1019.000000,126.12,2.52,57.0,11.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,81.500000,8.250000,0.0,77.250000,5.750000,1012.000000,793.61,15.87,25.0,14.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
5191,63.750000,1.250000,0.0,91.250000,6.000000,1012.750000,1943.16,38.86,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
5390,63.000000,9.600000,0.0,78.800000,6.000000,1015.400000,1103.33,22.07,32.0,6.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
860,48.000000,7.000000,0.0,80.333333,5.166667,1020.666667,1432.97,28.66,66.0,6.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()  # or MinMaxScaler()
X_train[cts_cols] = scaler.fit_transform(X_train[cts_cols])
X_valid[cts_cols] = scaler.fit_transform(X_valid[cts_cols])
X_test[cts_cols] = scaler.fit_transform(X_test[cts_cols])


In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6312 entries, 1867 to 7270
Columns: 146 entries, route_avg_temp to driving_style_proactive
dtypes: float64(143), int64(3)
memory usage: 7.1 MB


In [23]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [42]:
X_train['delay'] = y_train

2024-10-18 15:37:42,467 WARNING: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [49]:
corr_matrix = X_train.corr()

# Extract correlation of all features with the target
target_corr = corr_matrix['delay'].sort_values(ascending=False)

# Print the correlation matrix
# print("Correlation Matrix:\n", corr_matrix)

# Print correlation with target column
target_corr

delay                                      1.000000
is_midnight                                0.316212
distance                                   0.218548
average_hours                              0.218548
accident                                   0.122483
                                             ...   
origin_description_Light snow             -0.039844
origin_description_Patchy rain possible   -0.048887
destination_description_Sunny             -0.081680
avg_no_of_vehicles                        -0.114829
destination_visibility                    -0.131476
Name: delay, Length: 146, dtype: float64

In [26]:
import hsml
import mlflow
import mlflow.sklearn
import os
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import joblib  # For saving the model locally

# Define X_train, X_valid, y_train, y_valid (use your existing datasets)
X = X_train  # Assuming scaled features
y = y_train

# Split the data further for GridSearch if needed
# X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# Initialize MLflow
mlflow.set_experiment("ML Models with Hyperparameter Tuning")

# Initialize Hopsworks connection
connection = hsml.connection()  # Create a connection to Hopsworks
mr = connection.get_model_registry()  # Get the Hopsworks model registry

# Directory to save models temporarily
model_dir = "models/"

# Ensure the directory exists
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Define a function to train, log, and save models with MLflow and Hopsworks
def train_and_evaluate_model(model, param_grid, model_name):
    with mlflow.start_run(run_name=model_name):
        # GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)

        # Get the best model from GridSearch
        best_model = grid_search.best_estimator_

        # Predict on validation data
        y_pred = best_model.predict(X_valid)
        y_test_pred = best_model.predict(X_test)
        # Evaluate performance
        acc = accuracy_score(y_valid, y_pred)
        f1 = f1_score(y_valid, y_pred, average='weighted')

        acc_test = accuracy_score(y_test, y_test_pred)
        print(f"Test accuracy: {acc_test}")
        # Log parameters, metrics, and model with MLflow
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metrics({"accuracy": acc, "f1_score": f1})
        mlflow.sklearn.log_model(best_model, model_name)

        # Save the model locally
        local_model_path = f"{model_dir}{model_name}_model.pkl"
        joblib.dump(best_model, local_model_path)

        # Create a model instance for Hopsworks model registry
        model_instance = mr.python.create_model(
            name=model_name,
            metrics={"accuracy": acc, "f1_score": f1},
            description=f"{model_name} with hyperparameter tuning"
        )

        # Save the model to the Hopsworks Model Registry
        model_instance.save(local_model_path)

        # Print the results
        print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
        print(f"Accuracy: {acc}, F1 Score: {f1}")
        print(classification_report(y_valid, y_pred))

# Hyperparameter grids for each model
logreg_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [100, 200, 500]
}

rf_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]  # Address imbalance
}

xgb_params = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'colsample_bytree': [0.3, 0.7, 1],
    'subsample': [0.8, 1],
    # 'scale_pos_weight': [1, 3, 5]  # For class imbalance
}

# Train and evaluate models
# Train and evaluate models with corrected model names
train_and_evaluate_model(LogisticRegression(), logreg_params, "Logistic_Regression")
train_and_evaluate_model(RandomForestClassifier(), rf_params, "Random_Forest")
train_and_evaluate_model(XGBClassifier(use_label_encoder=False, eval_metric='logloss'), xgb_params, "XGBoost")



Connected. Call `.close()` to terminate connection gracefully.
Fitting 3 folds for each of 162 candidates, totalling 486 fits
2024-10-22 23:35:47,089 WARNING: FitFailedWarning: 
279 fits failed out of a total of 486.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rr010\OneDrive\Desktop\Class\Projects\TruckDelay_Classification\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\rr010\OneDrive\Desktop\Class\Projects\TruckDelay_Classification\.venv\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Us

2024/10/22 23:36:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Parameters for XGBoost: {'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1}
Accuracy: 0.7789923954372624, F1 Score: 0.7736475648406624
              precision    recall  f1-score   support

           0       0.81      0.87      0.84      1387
           1       0.71      0.59      0.65       717

    accuracy                           0.78      2104
   macro avg       0.76      0.73      0.74      2104
weighted avg       0.77      0.78      0.77      2104

